In [2]:
import networkx as nx
import graphviz
import numpy as np
from scipy.optimize import linprog
from math import floor
from time import time

In [ ]:
class TestBed():
    """
    We don't control the sense_schedule, it is what it is, our only way to interact with it is to query the bandwidth.
    (Although it will get updated when we add a rule)
    the only thing we control is the graph.
    """
    def __init__(self, sense_schedule):
        self._time_init = time()
        self._sense_schedule = sense_schedule
        self.testbed = nx.MultiGraph()

    @property
    def time(self):
        return time() - self._time_init

    @property
    def sense_schedule(self):
        # remove all entries where end_time has already passed
        self._sense_schedule = {k: v for k, v in self._sense_schedule.items() if k[3] > self.time}
        return self._sense_schedule

    def get_schedule(self):
        print("SENSE SCHEDULE: " + str(self.sense_schedule) + "\n")

    def add_site(self, site_name, port_capacity):
        self.testbed.add_node(site_name, port_capacity=port_capacity)
        return self

    ################################# DO NOT TOUCH #################################
    def _get_link_capacity(self, src, dst):
        return min(self.testbed.nodes[src]["port_capacity"], self.testbed.nodes[dst]["port_capacity"])

    def query_bandwidth(self, src, dst, time):
        """
        Ask for bandwidth between src and dst between now and now + time
        """
        link_cap = self._get_link_capacity(src, dst)
        # if there is already a request for this src, dst, we need to check if the time overlaps and subtract the reserved bandwidth
        node_reserved_bw = 0
        for (s_src, s_dst, s_start_time, s_end_time), s_reserved_bandwidth in self.sense_schedule.items():
            if (src == s_src or dst == s_src or src == s_dst or dst == s_dst):
                if self.time + time < s_start_time:
                    # no overlap
                    pass
                elif self.time > s_end_time:
                    # no overlap
                    pass
                else:
                    # there is an overlap
                    node_reserved_bw += s_reserved_bandwidth
        return max(0, link_cap - node_reserved_bw)
    ################################# DO NOT TOUCH #################################
    def update_graph(self):
        # remove all entries where end_time has already passed
        self.testbed.remove_edges_from([(u, v) for u, v, data in self.testbed.edges(data=True) if data['end_time'] < self.time])

    def fast_forward(self, seconds):
        # fast forward time
        self._time_init -= seconds
        print(f"Fast forwarding {seconds} seconds, Time now: {round(self.time)} \n")

    def draw(self):
        dot = graphviz.Graph()
        dot.attr(bgcolor="#ffffff", rankdir="LR", nodesep="1.5", ranksep="1.0")
        # self.update_graph()
        for node, data in self.testbed.nodes(data=True):
            dot.node(node, label=f"{node}\nCapacity: {data['port_capacity']}", style='filled', fillcolor='#ffcccb')
        for i, (u, v, data) in enumerate(self.testbed.edges(data=True)):
            # dot.edge(u, v, label=f"Rule ID: {i}\nPriority: {data['priority']}\nBandwidth: {data['bandwidth']}", fontsize='10')
            dot.edge(u, v, label=f"Rule ID: {i}\nPriority: {data['priority']}", fontsize='10')
        return dot

    def add_dummy_link(self, src, dst, priority):
        self.testbed.add_edge(src, dst, priority=priority)
        return self
    
    def get_used_bandwidth(self, site):
        self.update_graph()
        return sum([data['bandwidth'] for u, v, data in self.testbed.edges(site, data=True)])

    def add_rule(self, src, dst, priority, transfer_size):
        print("#"*20 + "ADDING RULE" + "#"*20)
        print(f"You are trying to add a rule which will take {transfer_size / self._get_link_capacity(src, dst)} seconds to finish on empty link of {self._get_link_capacity(src, dst)} Gbps")
        DEFAULT_TIME_WINDOW = 60
        MAX_TIME_WINDOW = 1600

        bandwidth_we_are_already_using = self.get_used_bandwidth(src) + self.get_used_bandwidth(dst)

        current_time_window = DEFAULT_TIME_WINDOW
        while current_time_window <= MAX_TIME_WINDOW:
            available_bandwidth = self.query_bandwidth(src, dst, current_time_window) + bandwidth_we_are_already_using
            if (transfer_size / available_bandwidth > current_time_window):
                current_time_window += DEFAULT_TIME_WINDOW
                continue
            else:
                break

        if current_time_window > MAX_TIME_WINDOW:
            print("Exceeded max time window")
            print("#"*16 + "FINISH ADDING RULE" + "#"*17 + "\n")
            return False
        elif available_bandwidth > 0:
            print(f"Adding rule from {src} to {dst} with bandwidth {available_bandwidth} for {transfer_size / available_bandwidth} seconds")
            # add the rule to the schedule
            self._sense_schedule[(src, dst, round(self.time), round(self.time + transfer_size / available_bandwidth))] = available_bandwidth
            # add the edge to the graph
            self.testbed.add_edge(src, dst, priority=priority, bandwidth=available_bandwidth, start_time=round(self.time), end_time=round(self.time + transfer_size / available_bandwidth))
            print("#"*16 + "FINISH ADDING RULE" + "#"*17 + "\n")
            return True
        else:
            return False

In [ ]:
schedule = { # {(src, dst, start_time, end_time), reserved_bandwidth}
    ("T2_US_UCSD", "T2_US_Caltech", 240, 360): 50,
    ("T2_US_Caltech", "T1_US_FNAL", 300, 400): 50,
}

tb = TestBed(schedule)

tb.add_site("T2_US_UCSD", 400)
tb.add_site("T2_US_Caltech", 400)
tb.add_site("T1_US_FNAL", 200)
tb.add_site("T2_US_Nebraska", 100)

tb.add_dummy_link("T2_US_UCSD", "T2_US_Caltech", 5)
tb.add_dummy_link("T2_US_Caltech", "T2_US_UCSD", 3)
tb.add_dummy_link("T2_US_Caltech", "T1_US_FNAL", 4)
tb.add_dummy_link("T1_US_FNAL", "T2_US_UCSD", 3)
tb.add_dummy_link("T1_US_FNAL", "T2_US_Nebraska", 2)

# dot = tb.draw()
# dot.render('output_graph', format='png', cleanup=True)

# tb.add_rule("T2_US_UCSD", "T2_US_Caltech", 5, 32000) # 320000 Gb = 40 TB
# tb.get_schedule()

# tb.fast_forward(100)
# tb.get_schedule()

# tb.add_rule("T2_US_UCSD", "T1_US_FNAL", 3, 64000)
# tb.get_schedule()

# # tb.fast_forward(1000)
# # tb.get_schedule()

# Old code

## We will try to solve the LP problem of maximizing $c^T x$ subject to $Ax \leq b$ &&  $x \geq 0$

In [36]:
def old_bandwidth_decision(G: nx.MultiGraph):
    simple_graph = nx.Graph()
    simple_graph.add_nodes_from(G.nodes(data=True))

    for u, v, data in G.edges(data=True):
        priority = data['priority']
        if simple_graph.has_edge(u, v):
            simple_graph[u][v]['priority'] += priority
        else:
            simple_graph.add_edge(u, v, priority=priority)

    nodes = list(simple_graph.nodes)
    edges = list(simple_graph.edges(data=True))

    n_nodes = len(nodes)
    n_edges = len(edges)

    c = np.zeros(n_edges)
    edge_index = {edge[:2]: i for i, edge in enumerate(edges)}
    for i, (u, v, data) in enumerate(edges):
        priority = data['priority']
        c[i] = -priority
    b = np.array([simple_graph.nodes[node]['port_capacity'] for node in nodes])
    A = nx.incidence_matrix(simple_graph, nodelist=nodes, edgelist=edges).toarray()

    print("=" * 30)
    print("A:", A)
    print("b:", b)
    print("c:", c)

    # what happens if we just try to solve this system of equations directly?
    print("=" * 30)
    print("Solving the system directly, i.e. x = A^-1 b:")
    print(np.linalg.solve(A, b))
    # this is not a feasible solution, we are not trying to just solve the system. We want to solve it subject to the constraints while minimizing the cost function.

    lower_bound = 0
    optim_result = None
    print("=" * 30)
    print("Optimization Results:")
    while True:
        bounds = [(lower_bound, None) for _ in range(len(edges))]
        curr_optim_result = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method='highs')
        if not curr_optim_result.success:
            break
        else:
            optim_result = curr_optim_result
            print(optim_result.x)
            lower_bound += 5
    bandwidths = optim_result.x
    print("=" * 30)
    print("Solution Chosen: ", bandwidths)
    edge_index = {edge[:2]: i for i, edge in enumerate(edges)}
    for u, v, key, data in G.edges(keys=True, data=True):
        total_priority = simple_graph[u][v]['priority']
        if total_priority > 0:
            proportion = data['priority'] / total_priority
            bandwidth = bandwidths[edge_index[(u, v)]] * proportion
            G[u][v][key]['bandwidth'] = floor(bandwidth)
        else:
            G[u][v][key]['bandwidth'] = 0

    print("=" * 30)
    print("Final Bandwidths:")
    for u, v, key, data in G.edges(keys=True, data=True):
        print(f"{u} -> {v} (priority: {data['priority']}, bandwidth: {data['bandwidth']})")